# <b>Querying a knowledge graph using python</b>

## 1. Set parameters for queries
This Tutorial is based on general action plans of cutting as shown in the presentation. The example scenario is based on the assumption, that a given robot is performing meal preparation tasks in a kitchen. It is given <b>two</b> parameters: <b>a task, and an object</b>. Currently, we are not looking at how these parameters are passed to the robot. Future work will extract these parameters from recipe instructions or natural language text, but right now we just pass them over to the robot. For doing this, we will use a simple dropdown menu in this file.

This step terefore will open a dropdown so you can choose parameters for the following queries. 

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
tasks = [('Cutting Action',"cut:CuttingAction"),
        ('Quartering', "cut:Quartering"),
        ('Julienning',"cut:Julienning"),
        ('Halving',"cut:Halving"),
        ('Dicing',"soma:Dicing"),
        ('Cutting',"soma:Cutting"),
        ('Slicing',"soma:Slicing"),
        ('Snipping',"cut:Snipping"),
        ('Slivering',"cut:Slivering"),
        ('Sawing',"cut:Sawing"),
        ('Paring',"cut:Paring"),
        ('Carving',"cut:Carving"),
        ('Mincing',"cut:Mincing"),
        ('Cubing',"cut:Cubing"),
        ('Chopping',"cut:Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
        ('apple', "obo:FOODON_03301710"),
        ('avocado', "obo:FOODON_00003600"),
        ('banana', "obo:FOODON_00004183"),
        ('bean', "obo:FOODON_03301403"),      
        ('cherry', "obo:FOODON_03301240"),
        ('citron', "obo:FOODON_03306596"),
        ('coconut', "obo:FOODON_00003449"),     
        ('cucumber', "obo:FOODON_00003415"),
        ('kiwi', "obo:FOODON_00004387"), 
        ('kumquat', "obo:FOODON_03306597"),
        ('lemon', "obo:FOODON_03301441"),
        ('lime', "obo:FOODON_00003661"),
        ('olive', "obo:FOODON_03317509"),
        ('orange', "obo:FOODON_03309832"),
        ('peach', "obo:FOODON_03315502"), 
        ('pepper', "obo:FOODON_00003520"),
        ('pineapple', "obo:FOODON_00003459"),
        ('pumpkin', "obo:FOODON_00003486"),
        ('strawberry', "obo:FOODON_00003443"),        
        ('squash', "obo:FOODON_00003539"),
        ('tomato', "obo:FOODON_03309927")]

task=""
tobject=""

# Takes a Task of the tasks list. 
# Then makes the previously initialized task variable global and assigns value of the given Task to it 
def chooseTask(Task):
    global task
    task=Task
    
# Takes an Object of the object list.
# Then makes the previously initialized tobject variable global and assigns value of the given Object to it 
def chooseObject(Object):
    global tobject
    tobject=Object
    
# Create the dropdown widgets
task_widget = widgets.Dropdown(options=tasks, description='Task:')
object_widget = widgets.Dropdown(options=objects, description='Object:')

# Define the event handlers
def taskEvent(event):
    chooseTask(event.new)

def objectEvent(event):
    chooseObject(event.new)
    
# Attach the event handlers to the widgets
task_widget.observe(taskEvent, names='value')
object_widget.observe(objectEvent, names='value')

# Combine the widgets using HBox
widgets_display = HBox([task_widget, object_widget])

# Display widgets
widgets_display

# optional: set Parameters manually:
#task = "cut:Quartering"
#tobject = "obo:FOODON_03301710"

## 2. import rdflib for querying the knowledge graph for action parameters
This step is needed to load libraries for execution of the queries

In [ ]:
import requests

!pip install rdflib
# import rdflib
from rdflib import Graph, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

## 3. Setup & parse the knowledge graph from its GitHub repo

In [ ]:
url = "https://raw.githubusercontent.com/Food-Ninja/CuttingFood/main/owl/food_cutting.owl"
response = requests.get(url)

g = Graph()
g.parse(data=response.content, format='xml')

# define prefixes to be used in the query 
CUT = Namespace("http://www.ease-crc.org/ont/food_cutting#")
OBO = Namespace("http://purl.obolibrary.org/obo/")
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
SIT_AWARE = Namespace("http://www.ease-crc.org/ont/situation_awareness#")

g.bind("owl", OWL)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("sit_aware", SIT_AWARE)
g.bind("cut", CUT)
g.bind("obo", OBO)

--------------------------------------------------------------------------------------------------------------------------------
# <b>Now let us start with querying the knowledge graph!</b>
## Query 1: We can query for the tool to be used for the chosen task:
With this query, a robot can determine which tool shall be used for the cutting task

In [ ]:
tool_query = f"""
SELECT ?res WHERE {{
    {tobject} rdfs:subClassOf* ?peel_dis.
    ?peel_dis owl:onProperty soma:hasDisposition.
    ?peel_dis owl:someValuesFrom ?peel_dis_vals.
    ?peel_dis_vals owl:intersectionOf ?afford_vals.
    ?afford_vals rdf:first sit_aware:Cuttability.
    ?afford_vals rdf:rest ?task_trigger.
    ?task_trigger rdf:rest ?trigger.
    ?trigger rdf:first ?trigger_wo_nil.
    ?trigger_wo_nil owl:onProperty soma:affordsTrigger.
    ?trigger_wo_nil owl:allValuesFrom ?trigger_tool.
    ?trigger_tool owl:allValuesFrom ?tool.
    BIND(REPLACE(STR(?tool), "^.*[#/]", "") AS ?res).
}}
"""

for row in g.query(tool_query):
    print(row.res)

# Query 2a: The robot needs to know if additional actions need to be executed:
Additional actions like peeling or core removal can be queried here. This helps the robot to determine if it can proceed with cutting or needs to perform a different task beforehand.

In [ ]:
additional_actions_query = f"""
SELECT ?action WHERE {{ 
  {tobject} rdfs:subClassOf ?restriction.
  ?restriction owl:onProperty cut:hasPart.
  ?restriction owl:someValuesFrom ?node.
  ?node owl:intersectionOf ?intersec.
  ?intersec rdf:first ?foodpart.
  ?intersec rdf:rest ?node2.
  ?node2 rdf:first ?part.
  ?part owl:onProperty cut:hasEdibility.
  {{?part owl:someValuesFrom cut:ShouldBeAvoided.}}
  UNION
  {{?part owl:someValuesFrom cut:MustBeAvoided.}}
  ?foodpart rdfs:subClassOf ?partrestriction.
  ?partrestriction owl:onProperty soma:hasDisposition.
  ?partrestriction owl:someValuesFrom ?partnode.
  ?partnode owl:intersectionOf ?intersection.
  ?intersection rdf:first ?disposition.
  ?intersection rdf:rest ?actionrestriction.
  ?actionrestriction rdf:first ?actionnode.
  ?actionnode owl:onProperty soma:affordsTask.
  ?actionnode owl:someValuesFrom ?action.
}}
"""

for row in g.query(additional_actions_query):
    print(row.action)

# Query 2b: The robot needs to know if prior actions need to be executed:
Prior actions like halving before quartering can be queried here. This helps the robot to determine how often it needs to perform motions.

In [ ]:
prior_action_query = f"""
SELECT ?res WHERE {{
    {task} rdfs:subClassOf* ?sub.
    ?sub owl:onProperty cut:requiresPriorTask .
    ?sub owl:someValuesFrom ?priortask.
    BIND(REPLACE(STR(?priortask), "^.*[#/]", "") AS ?res).
}}
"""

for row in g.query(prior_action_query):
    print(row.res)

# Query 3: The robot needs to also know the number of repetitions required for performing the task:
It is important to determine the needed number of repetitions in order to know when a task is finished.

In [ ]:
repetitions_query = f"""
SELECT ?res WHERE {{
    {{
        {task} rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {{
            ?rep_node owl:hasValue ?val.
            }}
        BIND("exactly 1" AS ?res)
    }}
    UNION
    {{
        {task} rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {{
            ?rep_node owl:minQualifiedCardinality ?val.
        }}
        BIND("at least 1" AS ?res)
    }}
}}
"""

for row in g.query(repetitions_query):
    print(row.res)

# Query 4: The position needed for action execution:
It is important to know where to cut an object.

In [ ]:
position_query = f"""
SELECT ?res WHERE {{
    {task} rdfs:subClassOf* ?pos_node.
    ?pos_node owl:onProperty cut:affordsPosition.
    ?pos_node owl:someValuesFrom ?pos.
    BIND(REPLACE(STR(?pos), "^.*[#/]", "") AS ?res).
}}
"""

for row in g.query(position_query):
    print(row.res)

# Query 5: What is the expected input form of the object that shall be cut:
When cutting a fruit, for some subtasks the robot does not cut the whole food object but a slice, or a stripe.

In [ ]:
input_query = f"""
SELECT ?res WHERE {{
    {{
        {task} rdfs:subClassOf* ?inter_node.
        ?inter_node owl:intersectionOf ?in_res_node.
        ?in_res_node rdf:first ?input_node.
        ?input_node owl:onProperty cut:hasInputObject.
        ?input_node owl:someValuesFrom ?target.
        FILTER NOT EXISTS {{
            ?target owl:unionOf ?union_node.
        }}
        BIND(REPLACE(STR(?target), "^.*[#/]", "") AS ?res).
    }}
    UNION
    {{
        {task} rdfs:subClassOf* ?inter_node.
        ?inter_node owl:intersectionOf ?in_res_node.
        ?in_res_node rdf:first ?input_node.
        ?input_node owl:onProperty cut:hasInputObject.
        ?input_node owl:someValuesFrom ?targets_node.
        ?targets_node owl:unionOf ?union_node.
        # Recursive traversal of rdf:first and rdf:rest
        {{
            ?union_node rdf:first ?target.
            BIND(REPLACE(STR(?target), "^.*[#/]", "") AS ?res).
        }}
        UNION
        {{
            ?union_node rdf:rest*/rdf:first ?target.
            BIND(REPLACE(STR(?target), "^.*[#/]", "") AS ?res).
        }}
    }}
}}
"""

for row in g.query(input_query):
    print(row.res)

# Query 6: What is the expected output form of the object that shall be cut:

In [ ]:
output_query = f"""
SELECT ?res ?cardinality WHERE {{
    {{
        {task} rdfs:subClassOf* ?inter_node.
        ?inter_node owl:intersectionOf ?in_res_node.
        ?in_res_node rdf:rest*/rdf:first ?result_node.
        ?result_node owl:onProperty cut:hasResultObject.
        # Handling "someValuesFrom" (no explicit cardinality)
        OPTIONAL {{
            ?result_node owl:someValuesFrom ?target.
            BIND("some" AS ?cardinality).
        }}
        # Handling "exactly X" constraints (qualified cardinality)
        OPTIONAL {{
            ?result_node owl:qualifiedCardinality ?card.
            ?result_node owl:onClass ?target.
            BIND(STR(?card) AS ?cardinality).
        }}
        # Handling "exactly X" constraints (non-qualified cardinality)
        OPTIONAL {{
            ?result_node owl:cardinality ?card.
            BIND(STR(?card) AS ?cardinality).
        }}
        BIND(REPLACE(STR(?target), "^.*[#/]", "") AS ?res).
    }}
}}
"""

for row in g.query(output_query):
    print(f'{row.cardinality}x {row.res}')